## Event Impact Modeling Code

In [ ]:
# -*- coding: utf-8 -*-
"""
Task 3: Event Impact Modeling
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Load data
df = pd.read_csv('../_data/processed/ethiopia_fi_enriched.csv')
observations = df[df['record_type'] == 'observation'].copy()
events = df[df['record_type'] == 'event'].copy()
impact_links = df[df['record_type'] == 'impact_link'].copy()

# Convert dates
observations['observation_date'] = pd.to_datetime(observations['observation_date'])
events['event_date'] = pd.to_datetime(events['event_date'])

# ============================================================================
# 1. PREPARE EVENT-IMPACT DATA
# ============================================================================

print("=" * 80)
print("EVENT IMPACT MODELING")
print("=" * 80)

# Merge impact links with events
event_impacts = pd.merge(
    impact_links,
    events[['event_id', 'event_name', 'event_date', 'category']],
    left_on='parent_id',
    right_on='event_id',
    how='left'
)

print(f"\n📊 Total impact relationships: {len(event_impacts)}")

# Summary by event
print("\n📋 Events and their impacts:")
for event_name in event_impacts['event_name'].unique():
    event_data = event_impacts[event_impacts['event_name'] == event_name]
    print(f"\n{event_name}:")
    for _, row in event_data.iterrows():
        print(f"  → {row['related_indicator']}: {row['impact_direction']} {row['impact_magnitude']}pp "
              f"after {row['lag_months']} months")

# ============================================================================
# 2. CREATE EVENT-INDICATOR MATRIX
# ============================================================================

# Get unique events and indicators
unique_events = event_impacts['event_name'].unique()
unique_indicators = event_impacts['related_indicator'].unique()

# Initialize matrix
impact_matrix = pd.DataFrame(
    index=unique_events,
    columns=unique_indicators,
    data=0.0
)

# Fill matrix with impact magnitudes
for _, row in event_impacts.iterrows():
    event = row['event_name']
    indicator = row['related_indicator']
    magnitude = row['impact_magnitude']
    
    # Adjust sign based on direction
    if row['impact_direction'] == 'positive':
        impact_matrix.loc[event, indicator] = magnitude
    elif row['impact_direction'] == 'negative':
        impact_matrix.loc[event, indicator] = -magnitude

print("\n" + "=" * 80)
print("EVENT-INDICATOR IMPACT MATRIX")
print("=" * 80)
print("\nImpact magnitude in percentage points:")
print(impact_matrix)

# ============================================================================
# 3. VISUALIZE IMPACT MATRIX
# ============================================================================

plt.figure(figsize=(12, 8))

# Create heatmap
mask = impact_matrix == 0
sns.heatmap(
    impact_matrix,
    annot=True,
    fmt='.2f',
    cmap='RdYlGn',
    center=0,
    mask=mask,
    linewidths=0.5,
    cbar_kws={'label': 'Impact (percentage points)'}
)

plt.title('Event-Impact Matrix', fontsize=16, fontweight='bold')
plt.xlabel('Financial Inclusion Indicators')
plt.ylabel('Events')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('../_data/processed/event_impact_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# 4. MODEL EVENT EFFECTS OVER TIME
# ============================================================================

def create_event_effect_vector(event_date, impact_magnitude, lag_months, 
                               total_months=60, effect_shape='gradual'):
    """
    Create a time series vector representing an event's effect over time.
    
    Parameters:
    - event_date: When the event occurred
    - impact_magnitude: Maximum effect size in percentage points
    - lag_months: Months until effect reaches maximum
    - total_months: Total time horizon to model
    - effect_shape: 'immediate', 'gradual', or 'delayed'
    
    Returns:
    - Array of effects for each month
    """
    
    if effect_shape == 'immediate':
        # Immediate effect, sustained
        effect = np.zeros(total_months)
        effect[lag_months:] = impact_magnitude
        
    elif effect_shape == 'gradual':
        # Gradual build-up to maximum
        effect = np.zeros(total_months)
        for i in range(lag_months, total_months):
            # Linear build-up over lag period
            if i < lag_months * 2:
                effect[i] = impact_magnitude * (i - lag_months + 1) / lag_months
            else:
                effect[i] = impact_magnitude
                
    elif effect_shape == 'delayed':
        # Delayed then sustained
        effect = np.zeros(total_months)
        effect[lag_months:] = impact_magnitude
        
    return effect

# ============================================================================
# 5. TEST AGAINST HISTORICAL DATA
# ============================================================================

print("\n" + "=" * 80)
print("HISTORICAL VALIDATION")
print("=" * 80)

# Get account ownership data
acc_data = observations[
    observations['indicator_code'].str.contains('ACC_OWNERSHIP', na=False)
].copy()
acc_data = acc_data.sort_values('observation_date')

# Convert to monthly time series (simplified)
acc_data['year'] = acc_data['observation_date'].dt.year
yearly_ownership = dict(zip(acc_data['year'], acc_data['value_numeric']))

# Test Telebirr impact (launched May 2021)
telebirr_impact = event_impacts[
    (event_impacts['event_name'].str.contains('Telebirr', na=False)) &
    (event_impacts['related_indicator'] == 'ACC_MM_ACCOUNT')
]

if not telebirr_impact.empty:
    print("\n📱 Telebirr Impact Analysis:")
    print(f"Event: {telebirr_impact['event_name'].iloc[0]}")
    print(f"Predicted impact: {telebirr_impact['impact_magnitude'].iloc[0]}pp "
          f"after {telebirr_impact['lag_months'].iloc[0]} months")
    
    # Actual change in mobile money accounts
    mm_2021 = 4.7  # From problem statement
    mm_2024 = 9.45  # From problem statement
    actual_change = mm_2024 - mm_2021
    
    print(f"Actual change (2021-2024): +{actual_change:.2f}pp")
    print(f"Predicted vs Actual difference: {telebirr_impact['impact_magnitude'].iloc[0] - actual_change:.2f}pp")
    
    if abs(telebirr_impact['impact_magnitude'].iloc[0] - actual_change) < 1:
        print("✅ Model prediction aligns well with actual data")
    else:
        print("⚠️ Model prediction differs from actual data")
        print("Possible reasons:")
        print("  - Other factors influenced growth")
        print("  - Impact magnitude needs adjustment")
        print("  - Lag period different than expected")

# ============================================================================
# 6. REFINE IMPACT ESTIMATES
# ============================================================================

print("\n" + "=" * 80)
print("IMPACT ESTIMATE REFINEMENT")
print("=" * 80)

# Calculate average impacts by event category
category_impacts = event_impacts.groupby('category').agg({
    'impact_magnitude': ['mean', 'std', 'count']
}).round(2)

print("\n📊 Average impacts by event category:")
print(category_impacts)

# Adjust estimates based on validation
refined_impacts = event_impacts.copy()

# Example adjustment: Increase Telebirr impact based on actual data
telebirr_mask = refined_impacts['event_name'].str.contains('Telebirr', na=False)
refined_impacts.loc[telebirr_mask & (refined_impacts['related_indicator'] == 'ACC_MM_ACCOUNT'), 
                    'impact_magnitude'] = 5.0  # Adjusted from maybe 4.0
refined_impacts.loc[telebirr_mask & (refined_impacts['related_indicator'] == 'ACC_MM_ACCOUNT'),
                    'confidence'] = 'high'

print("\n🔧 Refined impact estimates:")
for event_name in refined_impacts['event_name'].unique():
    event_data = refined_impacts[refined_impacts['event_name'] == event_name]
    if len(event_data) > 0:
        print(f"\n{event_name}:")
        for _, row in event_data.iterrows():
            print(f"  → {row['related_indicator']}: {row['impact_magnitude']}pp "
                  f"(confidence: {row['confidence']})")

# ============================================================================
# 7. CREATE DYNAMIC IMPACT MODEL
# ============================================================================

class EventImpactModel:
    """Model to simulate event impacts on financial inclusion indicators"""
    
    def __init__(self, base_year=2011, end_year=2027):
        self.base_year